In [1]:
import pandas as pd
import mysql.connector
import os

In [2]:
def export_all_tableau_data():
    """
    Export all data needed for Tableau dashboard - COMPLETE VERSION
    """
    conn = mysql.connector.connect(
        host='localhost',
        user='root',
        password='pass1234@',
        database='amazonSales'
    )
    
    print("="*60)
    print("EXPORTING TABLEAU DATASETS")
    print("="*60)
    
    # Create data directory if it doesn't exist
    os.makedirs('../data', exist_ok=True)
    
    # ===== 1. MAIN PRODUCT DATA =====
    print("\n1. Exporting main product data...")
    query_main = """
        SELECT 
            product_id,
            product_name,
            SUBSTRING_INDEX(category, '|', 1) as main_category,
            discounted_price,
            actual_price,
            discount_percentage,
            rating, 
            rating_count,
            (discounted_price * rating_count) as estimated_revenue,
            CASE 
                WHEN discounted_price < 10 THEN 'Budget'
                WHEN discounted_price < 50 THEN 'Economy'
                WHEN discounted_price < 150 THEN 'Mid-Range'
                WHEN discounted_price < 500 THEN 'Premium'
                ELSE 'Luxury'
            END as price_tier,
            CASE 
                WHEN discount_percentage < 25 THEN '0-25%'
                WHEN discount_percentage < 40 THEN '25-40%'
                WHEN discount_percentage < 60 THEN '40-60%'
                ELSE '60%+'
            END as discount_range,
            CASE 
                WHEN rating >= 4.5 THEN 'Excellent'
                WHEN rating >= 4.0 THEN 'High-Rated' 
                WHEN rating >= 3.0 THEN 'Average'
                ELSE 'Needs Improvement' 
            END as rating_category,
            ROUND(rating / discounted_price, 4) as value_score,
            (actual_price - discounted_price) as savings_amount
        FROM products
        WHERE discounted_price IS NOT NULL 
            AND rating IS NOT NULL
            AND rating_count IS NOT NULL
    """
    
    df_main = pd.read_sql(query_main, conn)
    df_main.to_csv('../data/tableau_main_products.csv', index=False)
    print(f"   ✅ tableau_main_products.csv ({len(df_main):,} rows)")
    
    # ===== 2. REVENUE BY TIER =====
    print("\n2. Exporting revenue by tier...")
    query_tier = """
        SELECT 
            CASE 
                WHEN discounted_price < 10 THEN 'Budget'
                WHEN discounted_price < 50 THEN 'Economy'
                WHEN discounted_price < 150 THEN 'Mid-Range'
                WHEN discounted_price < 500 THEN 'Premium'
                ELSE 'Luxury'
            END as price_tier,
            COUNT(*) as product_count,
            ROUND(AVG(rating), 2) as avg_rating,
            ROUND(AVG(discount_percentage), 1) as avg_discount,
            SUM(discounted_price * rating_count) as total_revenue,
            ROUND(AVG(rating_count), 0) as avg_reviews,
            ROUND(MIN(discounted_price), 2) as min_price,
            ROUND(MAX(discounted_price), 2) as max_price
        FROM products
        WHERE discounted_price IS NOT NULL AND rating IS NOT NULL
        GROUP BY price_tier
        ORDER BY MIN(discounted_price)
    """
    
    df_tier = pd.read_sql(query_tier, conn)
    df_tier['product_pct'] = (df_tier['product_count'] / df_tier['product_count'].sum() * 100).round(2)
    df_tier['revenue_pct'] = (df_tier['total_revenue'] / df_tier['total_revenue'].sum() * 100).round(2)
    
    # Add tier order for proper sorting in Tableau
    tier_order = {'Budget': 1, 'Economy': 2, 'Mid-Range': 3, 'Premium': 4, 'Luxury': 5}
    df_tier['tier_order'] = df_tier['price_tier'].map(tier_order)
    df_tier = df_tier.sort_values('tier_order')
    
    df_tier.to_csv('../data/tableau_revenue_by_tier.csv', index=False)
    print(f"   ✅ tableau_revenue_by_tier.csv ({len(df_tier)} rows)")
    
    # ===== 3. DISCOUNT EFFECTIVENESS =====
    print("\n3. Exporting discount effectiveness...")
    query_discount = """
        SELECT 
            CASE 
                WHEN discount_percentage < 25 THEN '0-25%'
                WHEN discount_percentage < 40 THEN '25-40%'
                WHEN discount_percentage < 60 THEN '40-60%'
                ELSE '60%+'
            END as discount_range,
            COUNT(*) as product_count,
            ROUND(AVG(rating), 2) as avg_rating,
            ROUND(AVG(rating_count), 0) as avg_review_count,
            ROUND(MIN(rating), 2) as min_rating,
            ROUND(MAX(rating), 2) as max_rating,
            SUM(discounted_price * rating_count) as total_revenue,
            ROUND(AVG(discounted_price), 2) as avg_price
        FROM products
        WHERE discount_percentage IS NOT NULL AND rating IS NOT NULL
        GROUP BY discount_range
        ORDER BY 
            CASE discount_range
                WHEN '0-25%' THEN 1
                WHEN '25-40%' THEN 2
                WHEN '40-60%' THEN 3
                ELSE 4
            END
    """
    
    df_discount = pd.read_sql(query_discount, conn)
    
    # Calculate lift metrics (KEY for your resume bullet!)
    baseline = df_discount[df_discount['discount_range'] == '0-25%'].iloc[0]
    df_discount['rating_lift_pct'] = ((df_discount['avg_rating'] / baseline['avg_rating'] - 1) * 100).round(2)
    df_discount['review_lift_multiple'] = (df_discount['avg_review_count'] / baseline['avg_review_count']).round(2)
    
    # Add range order
    range_order = {'0-25%': 1, '25-40%': 2, '40-60%': 3, '60%+': 4}
    df_discount['range_order'] = df_discount['discount_range'].map(range_order)
    
    df_discount.to_csv('../data/tableau_discount_effectiveness.csv', index=False)
    print(f"   ✅ tableau_discount_effectiveness.csv ({len(df_discount)} rows)")
    
    # Print key finding for resume
    optimal_range = df_discount.loc[df_discount['avg_rating'].idxmax()]
    print(f"   📊 KEY FINDING: {optimal_range['discount_range']} yields {optimal_range['rating_lift_pct']:.1f}% higher ratings")
    print(f"                   and {optimal_range['review_lift_multiple']:.1f}x more reviews")
    
    # ===== 4. CATEGORY PERFORMANCE =====
    print("\n4. Exporting category performance...")
    query_category = """
        SELECT 
            SUBSTRING_INDEX(category, '|', 1) as category,
            COUNT(*) as product_count,
            ROUND(AVG(rating), 2) as avg_rating,
            ROUND(AVG(discounted_price), 2) as avg_price,
            ROUND(AVG(discount_percentage), 1) as avg_discount,
            SUM(discounted_price * rating_count) as total_revenue,
            SUM(rating_count) as total_reviews,
            ROUND(MIN(discounted_price), 2) as min_price,
            ROUND(MAX(discounted_price), 2) as max_price
        FROM products
        WHERE discounted_price IS NOT NULL AND rating IS NOT NULL
        GROUP BY category
        HAVING product_count >= 10
        ORDER BY total_revenue DESC
        LIMIT 15
    """
    
    df_category = pd.read_sql(query_category, conn)
    df_category['revenue_pct'] = (df_category['total_revenue'] / df_category['total_revenue'].sum() * 100).round(2)
    df_category['avg_revenue_per_product'] = (df_category['total_revenue'] / df_category['product_count']).round(2)
    
    df_category.to_csv('../data/tableau_category_performance.csv', index=False)
    print(f"   ✅ tableau_category_performance.csv ({len(df_category)} rows)")
    
    # ===== 5. PRICE VS ENGAGEMENT (NEW - Important for scatter plot!) =====
    print("\n5. Exporting price vs engagement analysis...")
    query_engagement = """
        SELECT 
            discounted_price,
            rating_count,
            rating,
            discount_percentage,
            SUBSTRING_INDEX(category, '|', 1) as category,
            CASE 
                WHEN discounted_price < 10 THEN 'Budget'
                WHEN discounted_price < 50 THEN 'Economy'
                WHEN discounted_price < 150 THEN 'Mid-Range'
                WHEN discounted_price < 500 THEN 'Premium'
                ELSE 'Luxury'
            END as price_tier,
            (discounted_price * rating_count) as estimated_revenue
        FROM products
        WHERE discounted_price IS NOT NULL 
            AND rating IS NOT NULL
            AND rating_count >= 10  -- Filter out products with very few reviews
        ORDER BY RAND()
        LIMIT 5000  -- Sample for faster Tableau performance
    """
    
    df_engagement = pd.read_sql(query_engagement, conn)
    df_engagement.to_csv('../data/tableau_price_engagement.csv', index=False)
    print(f"   ✅ tableau_price_engagement.csv ({len(df_engagement):,} rows)")
    
    # ===== 6. TOP PERFORMERS (NEW - For highlighting success stories) =====
    print("\n6. Exporting top performers...")
    query_top = """
        SELECT 
            product_name,
            SUBSTRING_INDEX(category, '|', 1) as category,
            discounted_price,
            rating,
            rating_count,
            discount_percentage,
            (discounted_price * rating_count) as revenue,
            CASE 
                WHEN discounted_price < 10 THEN 'Budget'
                WHEN discounted_price < 50 THEN 'Economy'
                WHEN discounted_price < 150 THEN 'Mid-Range'
                WHEN discounted_price < 500 THEN 'Premium'
                ELSE 'Luxury'
            END as price_tier,
            ROUND(rating / discounted_price, 4) as value_score
        FROM products
        WHERE rating >= 4.0 
            AND rating_count >= 50
            AND discounted_price IS NOT NULL
        ORDER BY revenue DESC
        LIMIT 100
    """
    
    df_top = pd.read_sql(query_top, conn)
    df_top.to_csv('../data/tableau_top_performers.csv', index=False)
    print(f"   ✅ tableau_top_performers.csv ({len(df_top)} rows)")
    
    # ===== 7. KPI SUMMARY (NEW - For dashboard KPI cards) =====
    print("\n7. Calculating KPI summary...")
    query_kpi = """
        SELECT 
            COUNT(*) as total_products,
            COUNT(DISTINCT SUBSTRING_INDEX(category, '|', 1)) as total_categories,
            ROUND(SUM(discounted_price * rating_count), 2) as total_revenue,
            ROUND(AVG(rating), 2) as avg_rating,
            ROUND(AVG(discount_percentage), 1) as avg_discount,
            SUM(rating_count) as total_reviews,
            COUNT(CASE WHEN rating >= 4.0 THEN 1 END) as high_rated_count
        FROM products
        WHERE discounted_price IS NOT NULL 
            AND rating IS NOT NULL
    """
    
    df_kpi = pd.read_sql(query_kpi, conn)
    df_kpi['high_rated_pct'] = (df_kpi['high_rated_count'] / df_kpi['total_products'] * 100).round(2)
    df_kpi.to_csv('../data/tableau_kpi_summary.csv', index=False)
    print(f"   ✅ tableau_kpi_summary.csv (1 row with all KPIs)")
    
    # Print KPI summary
    kpi = df_kpi.iloc[0]
    print(f"\n   📊 KPI SUMMARY:")
    print(f"      Total Revenue: ${kpi['total_revenue']:,.2f}")
    print(f"      Total Products: {kpi['total_products']:,}")
    print(f"      Avg Rating: {kpi['avg_rating']:.2f} stars")
    print(f"      Avg Discount: {kpi['avg_discount']:.1f}%")
    
    conn.close()
    
    # ===== FINAL SUMMARY =====
    print("\n" + "="*60)
    print("✅ ALL DATASETS EXPORTED SUCCESSFULLY!")
    print("="*60)
    print(f"\nLocation: {os.path.abspath('../data/')}")
    print("\nFiles created:")
    print("  1. tableau_main_products.csv (Main product data)")
    print("  2. tableau_revenue_by_tier.csv (Revenue analysis)")
    print("  3. tableau_discount_effectiveness.csv (Discount insights)")
    print("  4. tableau_category_performance.csv (Category breakdown)")
    print("  5. tableau_price_engagement.csv (Scatter plot data)")
    print("  6. tableau_top_performers.csv (Success stories)")
    print("  7. tableau_kpi_summary.csv (Dashboard KPIs)")
    print("\n📊 Ready to import into Tableau Desktop!")
    print("\n💡 RECOMMENDED IMPORT ORDER:")
    print("   1. Start with tableau_kpi_summary.csv for KPI cards")
    print("   2. Then tableau_revenue_by_tier.csv for main charts")
    print("   3. Add tableau_main_products.csv for detailed analysis")
    print("   4. Use others for specific visualizations")

# Run the export
export_all_tableau_data()

EXPORTING TABLEAU DATASETS

1. Exporting main product data...
   ✅ tableau_main_products.csv (1,351 rows)

2. Exporting revenue by tier...
   ✅ tableau_revenue_by_tier.csv (5 rows)

3. Exporting discount effectiveness...
   ✅ tableau_discount_effectiveness.csv (4 rows)
   📊 KEY FINDING: 0-25% yields 0.0% higher ratings
                   and 1.0x more reviews

4. Exporting category performance...
   ✅ tableau_category_performance.csv (15 rows)

5. Exporting price vs engagement analysis...
   ✅ tableau_price_engagement.csv (1,338 rows)

6. Exporting top performers...
   ✅ tableau_top_performers.csv (100 rows)

7. Calculating KPI summary...
   ✅ tableau_kpi_summary.csv (1 row with all KPIs)

   📊 KPI SUMMARY:
      Total Revenue: $827,574,443.70
      Total Products: 1,351.0
      Avg Rating: 4.09 stars
      Avg Discount: 46.7%

✅ ALL DATASETS EXPORTED SUCCESSFULLY!

Location: /Users/bezatezera/Desktop/Data/amazonSales/Amazon_sales_analytics/project2/data

Files created:
  1. tableau_ma

/var/folders/8w/9hxx_8m540zcg2yy692kdy480000gn/T/ipykernel_94152/1828364424.py:59: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_main = pd.read_sql(query_main, conn)
/var/folders/8w/9hxx_8m540zcg2yy692kdy480000gn/T/ipykernel_94152/1828364424.py:87: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tier = pd.read_sql(query_tier, conn)
/var/folders/8w/9hxx_8m540zcg2yy692kdy480000gn/T/ipykernel_94152/1828364424.py:128: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_discount = pd.read_sql(query_discount, conn)
/var/folders/